In [1]:
import os
import os.path
import glob
import numpy as np
import Image,leargist
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
np.random.seed(1)

In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
X = np.zeros((num_samples, 320))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        im = Image.open(img_file)
        im1 = im.resize((64,64),Image.ANTIALIAS); # for faster computation
        des = leargist.color_gist(im1)
        X[cnt] = des[0:320]
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 320)

In [5]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [6]:
# Training top_model
C_range = np.logspace(-2, 3, 6)
gamma_range = np.logspace(-8, 1, 10)
param_grid = dict(gamma=gamma_range, C=C_range)

top_model = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(X,y)  # Training 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-08 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-07 .............................................
[CV] C=0.01, gamma=1e-06 .............................................
[CV] C=0.01, gamma=1e-06 .............................................
[CV] C=0.01, gamma=1e-06 .............................................
[CV] C=0.01, ga

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   21.9s


[CV] .............. C=0.01, gamma=1e-06, score=0.505089, total=  12.3s
[CV] .............. C=0.01, gamma=1e-06, score=0.505089, total=  12.4s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .............. C=0.01, gamma=1e-05, score=0.505089, total=  12.4s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] ............. C=0.01, gamma=0.0001, score=0.505089, total=  12.5s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .............. C=0.01, gamma=1e-05, score=0.505089, total=  12.8s
[CV] C=0.01, gamma=0.1 ...............................................
[CV] .............. C=0.01, gamma=1e-08, score=0.505718, total=  15.3s
[CV] C=0.01, gamma=0.1 ...............................................


[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   29.5s


[CV] .............. C=0.01, gamma=1e-08, score=0.505089, total=  16.3s
[CV] .............. C=0.01, gamma=1e-08, score=0.505089, total=  16.3s
[CV] C=0.01, gamma=0.1 ...............................................
[CV] .............. C=0.01, gamma=1e-08, score=0.505089, total=  16.3s
[CV] C=0.01, gamma=0.1 ...............................................
[CV] C=0.01, gamma=0.1 ...............................................
[CV] .............. C=0.01, gamma=1e-06, score=0.505089, total=  17.7s
[CV] C=0.01, gamma=1.0 ...............................................
[CV] .............. C=0.01, gamma=1e-07, score=0.505718, total=  17.5s
[CV] C=0.01, gamma=1.0 ...............................................
[CV] .............. C=0.01, gamma=1e-05, score=0.505089, total=  17.7s
[CV] C=0.01, gamma=1.0 ...............................................
[CV] .............. C=0.01, gamma=1e-05, score=0.505089, total=  17.8s
[CV] C=0.01, gamma=1.0 ...............................................
[CV] .

...

[CV] ............... C=100.0, gamma=1.0, score=0.970738, total=   3.4s
[CV] C=1000.0, gamma=1e-05 ...........................................
[CV] ............... C=100.0, gamma=0.1, score=0.914758, total=   5.4s
[CV] C=1000.0, gamma=1e-05 ...........................................
[CV] ............... C=100.0, gamma=1.0, score=0.945293, total=   3.4s
[CV] C=1000.0, gamma=1e-05 ...........................................
[CV] .............. C=100.0, gamma=10.0, score=0.959288, total=   5.2s
[CV] C=1000.0, gamma=1e-05 ...........................................
[CV] .............. C=100.0, gamma=10.0, score=0.969466, total=   5.1s
[CV] C=1000.0, gamma=0.0001 ..........................................
[CV] .............. C=100.0, gamma=10.0, score=0.956743, total=   5.1s
[CV] C=1000.0, gamma=0.0001 ..........................................
[CV] .............. C=100.0, gamma=10.0, score=0.973316, total=   7.5s
[CV] C=1000.0, gamma=0.0001 ..........................................
[CV] .

[Parallel(n_jobs=-1)]: Done 284 out of 300 | elapsed:  4.0min remaining:   13.4s


[CV] .............. C=1000.0, gamma=1.0, score=0.960560, total=   3.2s
[CV] ............ C=1000.0, gamma=1e-07, score=0.505089, total=  17.0s
[CV] ............ C=1000.0, gamma=1e-06, score=0.777354, total=  17.0s
[CV] .............. C=1000.0, gamma=1.0, score=0.936387, total=   2.9s
[CV] .............. C=1000.0, gamma=0.1, score=0.954198, total=   6.4s
[CV] .............. C=1000.0, gamma=0.1, score=0.921120, total=   5.7s
[CV] ............. C=1000.0, gamma=0.01, score=0.876590, total=   6.7s
[CV] .............. C=1000.0, gamma=0.1, score=0.922392, total=   5.6s
[CV] .............. C=1000.0, gamma=1.0, score=0.938931, total=   3.3s
[CV] ............. C=1000.0, gamma=10.0, score=0.956743, total=   3.5s
[CV] ............. C=1000.0, gamma=10.0, score=0.955471, total=   3.4s
[CV] ............. C=1000.0, gamma=10.0, score=0.959288, total=   4.2s
[CV] ............. C=1000.0, gamma=10.0, score=0.969466, total=   4.1s
[CV] ............. C=1000.0, gamma=10.0, score=0.973316, total=   4.5s


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.0min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03]), 'gamma': array([  1.00000e-08,   1.00000e-07,   1.00000e-06,   1.00000e-05,
         1.00000e-04,   1.00000e-03,   1.00000e-02,   1.00000e-01,
         1.00000e+00,   1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [7]:
top_model.cv_results_

{'mean_fit_time': array([ 12.22060184,  10.76435704,  10.85786843,  12.81355238,
         12.6994462 ,  10.10266666,   9.30368853,  12.77420578,
         10.85795541,  12.2447556 ,   8.96512198,   9.55591917,
         13.81289439,  14.91296434,  11.86368914,  10.14035378,
          9.51073842,   6.79855719,   4.75115042,   6.86737556,
         13.05988617,  11.80510597,  10.77646165,  11.40057855,
         12.4990716 ,   9.75670462,   7.46212735,   6.53496161,
          3.7378942 ,   4.86507034,  10.94399338,  11.78757977,
         13.1423296 ,  12.87319465,  11.73611612,   6.79733295,
          4.70751204,   5.07497082,   3.34784842,   4.56757498,
          9.95201626,  10.76175079,  14.0999434 ,  12.66360345,
          7.45540557,   4.68063502,   3.94729733,   3.48373017,
          2.80595398,   4.88071327,  11.18536558,  12.49414659,
         11.09188962,   7.77097621,   5.65234814,   4.03865643,
          4.24830356,   4.83982739,   2.73025861,   3.44850659]),
 'mean_score_time': a

In [8]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 1e-08},
 mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 9.9999999999999995e-08},
 mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 9.9999999999999995e-07},
 mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 1.0000000000000001e-05},
 mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 0.0001},
 mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 0.001},
 mean: 0.50521, std: 0.00025, params: {'C': 0.01, 'gamma': 0.01},
 mean: 0.71381, std: 0.01370, params: {'C': 0.01, 'gamma': 0.10000000000000001},
 mean: 0.83109, std: 0.01027, params: {'C': 0.01, 'gamma': 1.0},
 mean: 0.78046, std: 0.01076, params: {'C': 0.01, 'gamma': 10.0},
 mean: 0.50521, std: 0.00025, params: {'C': 0.10000000000000001, 'gamma': 1e-08},
 mean: 0.50521, std: 0.00025, params: {'C': 0.10000000000000001, 'gamma': 9.9999999999999995e-08},
 mean: 0.50521, std: 0.00025, params: {'C': 0.10000000000000001, 'gamma': 9.99999999999

In [9]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 10.0, 'gamma': 10.0} - score: 0.9631
